In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
heart_data = pd.read_csv("heart.csv")


In [3]:
heart_data.isnull().sum()


age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [4]:
heart_data.head(5)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [5]:
heart_data.shape
heart_data.describe()
print([x for x in heart_data.columns if heart_data[x].dtype != 'O']==heart_data.columns)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True]


In [6]:
X = heart_data.drop(['target'],axis=1)
X

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2


In [7]:
Y= heart_data['target']
Y

0       0
1       0
2       0
3       0
4       0
       ..
1020    1
1021    0
1022    0
1023    1
1024    0
Name: target, Length: 1025, dtype: int64

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection  import train_test_split,RandomizedSearchCV,GridSearchCV
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=10,shuffle=True)

X_train.shape,Y_test.shape

((820, 13), (205,))

In [9]:
heart_scaler =  StandardScaler()
X_train = heart_scaler.fit_transform(X_train)
X_test = heart_scaler.transform(X_test)
X_train

array([[ 1.03827659,  0.67755054,  0.99245979, ..., -0.60303869,
        -0.72619989,  1.07774559],
       [ 0.59936085,  0.67755054, -0.91790882, ...,  1.00768354,
         0.26297604,  1.07774559],
       [ 0.37990298,  0.67755054, -0.91790882, ..., -0.60303869,
         0.26297604,  1.07774559],
       ...,
       [ 1.03827659,  0.67755054,  1.9476441 , ..., -0.60303869,
        -0.72619989,  1.07774559],
       [ 0.70908979,  0.67755054, -0.91790882, ..., -0.60303869,
         0.26297604, -0.52615892],
       [ 0.37990298, -1.47590466,  0.99245979, ...,  1.00768354,
        -0.72619989, -0.52615892]], shape=(820, 13))

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier,GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,recall_score,precision_score,f1_score

In [11]:
def evaluate_model(y_true, y_pred):
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred)
    }


In [12]:
models ={
    'LogisticRegression':LogisticRegression(),
    "Xgboost":XGBClassifier(verbose=0),
    "DecisionTree":DecisionTreeClassifier(),
    "Adaboost" : AdaBoostClassifier(),
    "RandomForest" : RandomForestClassifier(),
    "KNeighbors" : KNeighborsClassifier()
}
models_list = []
score_test_list = []
score_train_list = []

for i in range(len(models)):
    model = list(models.values())[i]

    model.fit(X_train,Y_train)
    Y_train_pred = model.predict(X_train)
    Y_test_pred = model.predict(X_test)
    score_train,recall_train,precision_train,f1_train = evaluate_model(Y_train,Y_train_pred)
    score_test,recall_test,precision_test,f1_test = evaluate_model(Y_test,Y_test_pred)


    models_list.append(list(models.keys())[i])
    print(list(models.keys())[i])
    print("Model Performance For Training Set")
    print("accuracy_score:",score_train)
    print("recall_score:",recall_train)
    print("precision_score:",precision_train)
    print(confusion_matrix(Y_train,Y_train_pred))
    score_train_list.append(score_train)
    print('--' * 50)
    print("Model Performance For Test Set")
    print("accuracy_score:",score_test)
    print("recall_score:",recall_test)
    print("precision_score:",precision_test)
    print(confusion_matrix(Y_test,Y_test_pred))
    print('==' * 50)
    score_test_list.append(score_test)






LogisticRegression
Model Performance For Training Set
accuracy_score: accuracy
recall_score: recall
precision_score: precision
[[324  75]
 [ 42 379]]
----------------------------------------------------------------------------------------------------
Model Performance For Test Set
accuracy_score: accuracy
recall_score: recall
precision_score: precision
[[78 22]
 [ 8 97]]
Xgboost
Model Performance For Training Set
accuracy_score: accuracy
recall_score: recall
precision_score: precision
[[399   0]
 [  0 421]]
----------------------------------------------------------------------------------------------------
Model Performance For Test Set
accuracy_score: accuracy
recall_score: recall
precision_score: precision
[[100   0]
 [  0 105]]
DecisionTree
Model Performance For Training Set
accuracy_score: accuracy
recall_score: recall
precision_score: precision
[[399   0]
 [  0 421]]
----------------------------------------------------------------------------------------------------
Model Performa

c:\projects\MultiDiseaseClassification\DisClf\Lib\site-packages\xgboost\training.py:183: UserWarning: [01:11:06] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "verbose" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Adaboost
Model Performance For Training Set
accuracy_score: accuracy
recall_score: recall
precision_score: precision
[[333  66]
 [ 35 386]]
----------------------------------------------------------------------------------------------------
Model Performance For Test Set
accuracy_score: accuracy
recall_score: recall
precision_score: precision
[[84 16]
 [ 7 98]]
RandomForest
Model Performance For Training Set
accuracy_score: accuracy
recall_score: recall
precision_score: precision
[[399   0]
 [  0 421]]
----------------------------------------------------------------------------------------------------
Model Performance For Test Set
accuracy_score: accuracy
recall_score: recall
precision_score: precision
[[100   0]
 [  0 105]]
KNeighbors
Model Performance For Training Set
accuracy_score: accuracy
recall_score: recall
precision_score: precision
[[376  23]
 [ 21 400]]
----------------------------------------------------------------------------------------------------
Model Performance For

In [13]:
param_grid = {
    'LogisticRegression': {
        'model': LogisticRegression(max_iter=1000),
        'params': {
            'C': [0.01, 0.1, 1, 10],
            'solver': ['liblinear', 'lbfgs']
        }
    },
    'AdaBoost': {
        'model': AdaBoostClassifier(),
        'params': {
            'n_estimators': [50, 100, 150],
            'learning_rate': [0.01, 0.1, 1]
        }
    },
    'RandomForest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [100, 200],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5]
        }
    },
    'KNeighbors': {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3, 5, 7, 9],
            'weights': ['uniform', 'distance'],
            'metric': ['euclidean', 'manhattan']
        }
    },
    'XGBoost': {
        'model': XGBClassifier(use_label_encoder=False, eval_metric='logloss', verbosity=0),
        'params': {
            'n_estimators': [50, 100],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1, 0.2]
        }
    }
}

best_models = {}


for name,mp in param_grid.items():
    print(f"tuning {name}")
    grid = GridSearchCV(estimator=mp['model'],param_grid=mp['params'],cv=5,n_jobs=-1,scoring='accuracy')
    grid.fit(X_train,Y_train)
    print(f"Best params for {name}: {grid.best_params_}")
    print(f"Best cross-validation accuracy: {grid.best_score_:.4f}")
    print("="*60)
    best_models[name] = grid.best_estimator_


for name, model in best_models.items():
    y_pred = model.predict(X_test)
    print(f"Evaluation for {name}:")
    metrics = evaluate_model(Y_test, y_pred)
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")
    print("-" * 40)



tuning LogisticRegression
Best params for LogisticRegression: {'C': 1, 'solver': 'liblinear'}
Best cross-validation accuracy: 0.8463
tuning AdaBoost
Best params for AdaBoost: {'learning_rate': 1, 'n_estimators': 150}
Best cross-validation accuracy: 0.8902
tuning RandomForest
Best params for RandomForest: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}
Best cross-validation accuracy: 0.9854
tuning KNeighbors
Best params for KNeighbors: {'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'distance'}
Best cross-validation accuracy: 0.9902
tuning XGBoost
Best params for XGBoost: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}
Best cross-validation accuracy: 0.9829
Evaluation for LogisticRegression:
accuracy: 0.8537
recall: 0.9238
precision: 0.8151
f1: 0.8661
----------------------------------------
Evaluation for AdaBoost:
accuracy: 0.9122
recall: 0.9333
precision: 0.8991
f1: 0.9159
----------------------------------------
Evaluation for RandomForest:
accuracy

In [14]:
heart_model = AdaBoostClassifier(learning_rate=1,n_estimators= 150)
heart_model.fit(X_train,Y_train)
Y_test = heart_model.predict(X_test)

Y_test


array([0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 0])

In [15]:
import os


In [ ]:

# models_path = "./saved_models"
# os.makedirs(models_path,exist_ok=True)

In [17]:
import pickle
os.makedirs("models", exist_ok=True)

with open("models/heart_model.pkl", 'wb') as file:
    pickle.dump(heart_model, file)

with open("models/heart_scaler.pkl", 'wb') as file:
    pickle.dump(heart_scaler, file)